# Decision Tree Classifier  
Tutorial Link: https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/


### Gini Index

A Gini score gives an idea of how good a split is by how mixed the classes are in the two groups created by the split. A perfect separation results in a Gini score of 0, whereas the worst case split that results in 50/50 classes in each group result in a Gini score of 0.5 (for a 2 class problem).  

e.g.  
proportion = count(class_value) / count(rows)  
Proportions ->  
    group_1_class_0 = 2 / 2 = 1  
    group_1_class_1 = 0 / 2 = 0  
    group_2_class_0 = 0 / 2 = 0  
    group_2_class_1 = 2 / 2 = 1  
    
Gini Index calculation ->  
gini_index = sum(proportion * (1.0 - proportion))  
gini_index = 1.0 - sum(proportion * proportion)  

The Gini index for each group must then be weighted by the size of the group, relative to all of the samples in the parent:  

gini_index = (1.0 - sum(proportion * proportion)) * (group_size/total_samples)  

Gini(group_1) = (1 - (1*1 + 0*0)) * 2/4  
Gini(group_1) = 0.0  

Gini(group_2) = (1 - (0*0 + 1*1)) * 2/4  
Gini(group_2) = 0.0  

In [1]:
#Calculate the Gini index for a split dataset
def gini_index(groups, classes):
    # count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    # sum weighted Gini index for each group
    gini = 0.0
    for group in groups:
        size = float(len(group))
        # avoid divide by zero 
        if size == 0:
            continue
        score = 0.0
        # score the group based on the score for each class
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        gini += (1.0 - score) * (size / n_instances)
    return gini

In [2]:
# test Gini values
print(gini_index([[[1, 1], [1, 0]], [[1, 1], [1, 0]]], [0, 1]))
print(gini_index([[[1, 0], [1, 0]], [[1, 1], [1, 1]]], [0, 1]))

0.5
0.0


### Splitting a Dataset  
Splitting a dataset means separating a dataset into two lists of rows given the index of an attribute and a split value for that attribute. Once we have the two groups, we can then use our Gini score above to evaluate the cost of the split.  

In [3]:
#Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

### Evaluating All Splits  
With the Gini function above and the test split function we now have everything we need to evaluate splits. Given a dataset, we must check every value on each attribute as a candidate split, evaluate the cost of the split and find the best possible split we could make.  

Once the best split is found, we can use it as a node in our decision tree.

In [4]:
# Select the best split for a dataset 
def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            #print('X%d < %.3f Gini=%.3f' % ((index+1), row[index], gini))
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index,'value':b_value,'groups':b_groups}


In [5]:
dataset = [[2.771244718,1.784783929,0],
 [1.728571309,1.169761413,0],
 [3.678319846,2.81281357,0],
 [3.961043357,2.61995032,0],
 [2.999208922,2.209014212,0],
 [7.497545867,3.162953546,1],
 [9.00220326,3.339047188,1],
 [7.444542326,0.476683375,1],
 [10.12493903,3.234550982,1],
 [6.642287351,3.319983761,1]]
split = get_split(dataset)
print('Split: [X%d < %.3f]' % ((split['index']+1), split['value']))

Split: [X1 < 6.642]


### Build a Tree  

We call the above get_split() function using the entire dataset.  
Adding more nodes to our tree is more interesting.  
Building a tree may be divided into 3 main parts:  

1. Terminal Nodes.  
2. Recursive Splitting.  
3. Building a Tree.  

#### (1) Terminal Nodes  
We need to decide when to stop growing a tree.  

We can do that using the depth and the number of rows that the node is responsible for in the training dataset.  

**Maximum Tree Depth**. This is the maximum number of nodes from the root node of the tree. Once a maximum depth of the tree is met, we must stop splitting adding new nodes. Deeper trees are more complex and are more likely to overfit the training data.  

**Minimum Node Records**. This is the minimum number of training patterns that a given node is responsible for. Once at or below this minimum, we must stop splitting and adding new nodes. Nodes that account for too few training patterns are expected to be too specific and are likely to overfit the training data.  

In [6]:
# Create a terminal node value
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes),key=outcomes.count)

#### (2) Recursive Splitting  
We know how and when to create terminal nodes, now we can build our tree.  

Building a decision tree involves calling the above developed get_split() function over and over again on the groups created for each node.  

New nodes added to an existing node are called child nodes. A node may have zero children (a terminal node), one child (one side makes a prediction directly) or two child nodes. We will refer to the child nodes as left and right in the dictionary representation of a given node.  

Once a node is created, we can create child nodes recursively on each group of data from the split by calling the same function again.  

In [7]:
# Create child splits for a node or make terminal
def split(node,max_depth,min_size,depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # Check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'],max_depth,min_size,depth+1)

#### (3) Building a Tree  
Building the tree involves creating the root node and calling the split() function that then calls itself recursively to build out the whole tree.  

In [13]:
# Build a decision tree
def build_tree(train, max_depth, min_size):
    root = get_split(train)
    print('root before split:',root)
    split(root,max_depth, min_size, 1)
    print('root after split:',root)
    return root

In [14]:
# Print a decision tree
def print_tree(node, depth=0):
    if isinstance(node,dict):
        print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print('%s[%s]' % ((depth*' ', node)))

In [15]:
tree = build_tree(dataset, 1, 1)
print_tree(tree)

root before split: {'index': 0, 'value': 6.642287351, 'groups': ([[2.771244718, 1.784783929, 0], [1.728571309, 1.169761413, 0], [3.678319846, 2.81281357, 0], [3.961043357, 2.61995032, 0], [2.999208922, 2.209014212, 0]], [[7.497545867, 3.162953546, 1], [9.00220326, 3.339047188, 1], [7.444542326, 0.476683375, 1], [10.12493903, 3.234550982, 1], [6.642287351, 3.319983761, 1]])}
root after split: {'index': 0, 'value': 6.642287351, 'left': 0, 'right': 1}
[X1 < 6.642]
 [0]
 [1]


### Making a Prediction  
Making predictions with a decision tree involves navigating the tree with the specifically provided row of data.  

Again, we can implement this using a recursive function, where the same prediction routine is called again with the left or the right child nodes, depending on how the split affects the provided data.  

We must check if a child node is either a terminal value to be returned as the prediction, or if it is a dictionary node containing another level of the tree to be considered.  


In [11]:
# Make a prediction with a decision tree
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

In [12]:
#  predict with a stump
stump = {'index': 0, 'right': 1, 'value': 6.642287351, 'left': 0}
for row in dataset:
    prediction = predict(stump, row)
    print('Expected=%d, Got=%d' % (row[-1], prediction))

Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1


### Banknote Dataset

We will evaluate the algorithm using k-fold cross-validation with 5 folds. This means that 1372/5=274.4 or just over 270 records will be used in each fold. We will use the helper functions evaluate_algorithm() to evaluate the algorithm with cross-validation and accuracy_metric() to calculate the accuracy of predictions.

In [13]:
# CART on the Bank Note dataset
from random import seed
from random import randrange
from csv import reader

# Load a CSV file
def load_csv(filename):
    file = open(filename, "rt")
    lines = reader(file)
    dataset = list(lines)
    return dataset

# Convert string column to float
def str_column_to_float(dataset,column):
    for row in dataset:
        row[column] = float(row[column].strip())

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

def evaluate_algorithm(dataset,algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores
            
            

In [14]:
#Classification and Regression Tree Algorithm
def decision_tree(train, test, max_depth, min_size):
    tree = build_tree(train, max_depth, min_size)
    predictions = list()
    for row in test:
        prediction = predict(tree, row)
        predictions.append(prediction)
    return predictions

In [15]:
# Test CART on Bank Note dataset
seed(1)
# load and prepare data
filename = 'data_banknote_authentication.csv'
dataset = load_csv(filename)
# convert string attributes to integers
for i in range(len(dataset[0])):
    str_column_to_float(dataset, i)
# evaluate algorithm
n_folds = 5
max_depth = 5
min_size = 10
scores = evaluate_algorithm(dataset, decision_tree, n_folds, max_depth, min_size)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [96.35036496350365, 97.08029197080292, 97.44525547445255, 98.17518248175182, 97.44525547445255]
Mean Accuracy: 97.299%


---

In [1]:
from sklearn.datasets import make_regression
X, y = make_regression(n_samples=1300, n_features=2, noise=1, random_state=42)

In [2]:
import numpy as np

In [3]:
X[:10]

array([[ 2.40341559, -0.0576188 ],
       [ 1.61222063,  0.89683932],
       [-1.68343819, -0.80587007],
       [ 1.14282281,  0.75193303],
       [-2.42424026,  0.8840454 ],
       [-0.79252074, -0.11473644],
       [-0.30803428,  0.77966053],
       [ 0.93567839,  1.27155509],
       [ 1.50075979,  0.85022174],
       [-0.25879606,  1.59864717]])

In [76]:
for i in range(len(y)):
    y[i] = np.array(y[i])

In [9]:
data = [[X[i,0],X[i,1],y[i]] for i in range(len(X))]

In [10]:
data[0]

[2.403415585238275, -0.05761879703358539, 99.76438806004]

In [8]:
test = [np.concatenate((X[i],[y[i]])) for i in range(len(X))]

In [11]:
test[0]

array([ 2.40341559e+00, -5.76187970e-02,  9.97643881e+01])

In [15]:
test2 = list(zip(X,y))

In [16]:
test2[0]

(array([ 2.40341559, -0.0576188 ]), 99.76438806004)

In [21]:
transform_y = [[y[i]] for i in range(len(y))]


In [22]:
test3 = np.concatenate((X,transform_y),axis=1)

In [23]:
test3[0]

array([ 2.40341559e+00, -5.76187970e-02,  9.97643881e+01])

In [12]:
data[0][-1] - test[0][-1]

0.0

In [113]:
def MSE(groups):
    mse = 0.0
    for group in groups:
        size = float(len(group))
        if size == 0:
            continue
        y_hat = sum([row[-1] for row in group]) / size
        mse += sum([(row[-1] - y_hat)**2 for row in group]) / size 
    return mse

In [114]:
import math as m

In [115]:
# Select the best split for a dataset 
def get_split(dataset):
    b_index, b_value, b_score, b_groups = m.inf, m.inf, m.inf, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            mse = MSE(groups)
            #print('X%d < %.3f MSE=%.3f' % ((index+1), row[index], mse))
            if mse < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], mse, groups
    return {'index':b_index,'value':b_value,'groups':b_groups}

In [116]:
split = get_split(data[:10])
print('Split: [X%d < %.3f]' % ((split['index']+1), split['value']))

Split: [X1 < -0.308]


In [117]:
print(data[:10])

[[2.403415585238275, -0.05761879703358539, 99.76438806004], [1.6122206282554243, 0.8968393158655319, 159.639496052825], [-1.6834381922209503, -0.8058700664961888, -153.4926722547169], [1.1428228145150205, 0.7519330326867741, 125.07288207637336], [-2.4242402602729416, 0.8840453963610497, -15.738088311279796], [-0.7925207384327007, -0.11473644146689901, -43.840731443647435], [-0.30803428418574125, 0.7796605322693398, 65.07288857599129], [0.9356783931474612, 1.2715550949941588, 166.96381939017905], [1.5007597906343109, 0.8502217421134929, 149.56874045073454], [-0.25879606266710237, 1.598647170504717, 147.08460389363734]]


In [118]:
bin_1, bin_2 = [], []
for x in data[:10]:
    if x[-1] < -0.308:
        bin_1.append(x)
    else:
        bin_2.append(x)
print(bin_1)
print(bin_2)

[[-1.6834381922209503, -0.8058700664961888, -153.4926722547169], [-2.4242402602729416, 0.8840453963610497, -15.738088311279796], [-0.7925207384327007, -0.11473644146689901, -43.840731443647435]]
[[2.403415585238275, -0.05761879703358539, 99.76438806004], [1.6122206282554243, 0.8968393158655319, 159.639496052825], [1.1428228145150205, 0.7519330326867741, 125.07288207637336], [-0.30803428418574125, 0.7796605322693398, 65.07288857599129], [0.9356783931474612, 1.2715550949941588, 166.96381939017905], [1.5007597906343109, 0.8502217421134929, 149.56874045073454], [-0.25879606266710237, 1.598647170504717, 147.08460389363734]]


In [119]:
# Create a terminal node value
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return sum(outcomes)/len(outcomes)

In [120]:
# Create child splits for a node or make terminal
def split(node,max_depth,min_size,depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # Check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'],max_depth,min_size,depth+1)

In [121]:
#Classification and Regression Tree Algorithm
def decision_tree(train, test, max_depth, min_size):
    tree = build_tree(train, max_depth, min_size)
    predictions = list()
    for row in test:
        prediction = predict(tree, row)
        predictions.append(prediction)
    return predictions

In [130]:
# Calculate MSE
def accuracy_metric(actual, predicted):
    total = 0
    for i in range(len(actual)):
        total += (actual[i] - predicted[i])**2 
    return total / float(len(actual))

In [129]:
# evaluate algorithm
n_folds = 5
max_depth = 5
min_size = 10
scores = evaluate_algorithm(data, decision_tree, n_folds, max_depth, min_size)
print('Scores: %s' % scores)
print('AVG MSE: %.3f' % (sum(scores)/float(len(scores))))

Scores: [4635.889088230275, 5511.951224805913, 5386.04266248655, 5591.2031250409445, 5572.4239600926]
AVG MSE: 5339.502
